# Dadbot: Dad's memorial bot based on RASA (old style)

## Starting Jupyter Notebook with necessary imports

In [1]:
%matplotlib inline

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')


# Installations
* Rasa
* SpaCy Language Model

In [2]:
import sys
python = sys.executable

# In your environment run:
#!{python} -m pip install -U pip
#!{python} -m pip install tensorflow[tensorflow-addons]==2.1.0
#!{python} -m pip install rasa[convert]~=1.10.0

In [3]:
!{python} -m spacy download es_core_news_md

You should consider upgrading via the '/home/debian/.conda/envs/rasa/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_md')


## Downloading the Spanish Language Model

In [4]:
!{python} -m spacy link es_core_news_md es --force;

✔ Linking successful
/home/debian/.conda/envs/rasa/lib/python3.7/site-packages/es_core_news_md -->
/home/debian/.conda/envs/rasa/lib/python3.7/site-packages/spacy/data/es
You can now load the model via spacy.load('es')


## Import the Installations

In [5]:
import rasa


# 1. Teaching the bot to understand user inputs using Rasa NLU

## Training the NLU Model.

In [6]:
from rasa.model import get_model
from rasa.nlu import config, training_data, utils
from rasa.nlu.utils import write_to_file
from rasa.nlu.components import ComponentBuilder
from rasa.nlu.config import RasaNLUModelConfig
from rasa.nlu.model import Interpreter, Trainer, TrainingData
from rasa.nlu.components import Component
from rasa.nlu.tokenizers.tokenizer import Token
from rasa.utils.tensorflow.constants import ENTITY_RECOGNITION

import spacy

#spacy_parser = spacy.load('es_core_news_md')
#nlp = spacy.load('es')

# loading the nlu training samples
training_data = training_data.loading.load_data("data/nlu/nlu-papaito.md")

# trainer to train our pipeline
trainer = Trainer(config.load("config_DIET.yml"))

# train the model!
interpreter = trainer.train(training_data)

# store it for future use
model_directory = trainer.persist("./models/nlu", fixed_model_name="current")

INFO:rasa.nlu.model:Starting to train component WhitespaceTokenizer
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component RegexFeaturizer
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component LexicalSyntacticFeaturizer
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component CountVectorsFeaturizer
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component CountVectorsFeaturizer
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component DIETClassifier
Epochs: 100%|██████████| 100/100 [00:57<00:00,  1.74it/s, t_loss=2.078, i_loss=0.096, entity_loss=0.036, i_acc=0.990, entity_f1=0.868]
INFO:rasa.utils.tensorflow.models:Finished training.
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component EntitySynonymMapper
INFO:rasa.nlu.model:Finished training 

## Evaluating the NLU model on a random text

In [7]:
# A helper function for prettier output

def pprint(o):   
    print(json.dumps(o, indent=2))
    
pprint(interpreter.parse("dejándome el coche"))

{
  "intent": {
    "name": "ask_help",
    "confidence": 0.999729573726654
  },
  "entities": [],
  "intent_ranking": [
    {
      "name": "ask_help",
      "confidence": 0.999729573726654
    },
    {
      "name": "ask_comida",
      "confidence": 0.0002008982264669612
    },
    {
      "name": "mood_deny",
      "confidence": 2.6700656235334463e-05
    },
    {
      "name": "ask_ultimas_noticias",
      "confidence": 1.7241558452951722e-05
    },
    {
      "name": "ask_cuenta_algo",
      "confidence": 1.065205015038373e-05
    },
    {
      "name": "mood_affirm",
      "confidence": 4.700673798652133e-06
    },
    {
      "name": "ask_deporte_futbol",
      "confidence": 4.559474291454535e-06
    },
    {
      "name": "ask_razon",
      "confidence": 2.234985686300206e-06
    },
    {
      "name": "mood_unhappy",
      "confidence": 1.8329972135688877e-06
    },
    {
      "name": "ask_ocupacion",
      "confidence": 1.5469159961867263e-06
    }
  ],
  "response_selector

## Evaluating the NLU model on a test data
(Here we are using the data at hand i.e nlu.md but it isr recommended to use unseen data)

In [8]:
from rasa.nlu.test import run_evaluation

run_evaluation("data/nlu/nlu-papaito.md", model_directory)

Instructions for updating:
Use tf.identity instead.
INFO:rasa.nlu.test:Running model for predictions:
100%|██████████| 612/612 [00:03<00:00, 186.65it/s]
INFO:rasa.nlu.test:Intent evaluation results:
INFO:rasa.nlu.test:Intent Evaluation: Only considering those 612 examples that have a defined intent out of 612 examples
INFO:rasa.nlu.test:F1-Score:  0.9885992061947165
INFO:rasa.nlu.test:Precision: 0.9889797294209058
INFO:rasa.nlu.test:Accuracy:  0.988562091503268
INFO:rasa.nlu.test:Classification report: 
                      precision    recall  f1-score   support

            ask_help       1.00      1.00      1.00        38
             goodbye       1.00      0.94      0.97        33
          mood_happy       0.98      1.00      0.99        51
ask_ultimas_noticias       1.00      1.00      1.00        22
          ask_gustos       1.00      1.00      1.00         4
          ask_comida       1.00      0.96      0.98        25
           mood_deny       1.00      0.95      0.97     

{'intent_evaluation': {'predictions': [{'text': '¡ey!',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9997880458831787},
   {'text': '¡eyyy!',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9999029636383057},
   {'text': 'hola tú',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9998936057090759},
   {'text': 'hola',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9999706745147705},
   {'text': 'hola de nuevo',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9999651312828064},
   {'text': 'buenos días',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9999770522117615},
   {'text': 'buenos días por la mañana',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9999502897262573},
   {'text': 'buenas tardes',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9595496654510498},
   {'text': 'buenas tardes por las tardes',
    '

# 2. Teaching the bot to respond using Rasa Core

##  Visualising the Training Data

In [9]:
#!apt-get -qq install -y graphviz libgraphviz-dev pkg-config;
#!breq install graphviz

#!conda install -y -n rasa pygraphviz pkg-config;

In [10]:
from IPython.display import Image
from rasa.core.agent import Agent
 
agent = Agent('domain-papaito.yml')
#agent.visualize("data/core/stories-papaito.md", "story_graph.png", max_history=2)
#Image(filename="story_graph.png")

## Training a Dialogue Model

In [11]:
from rasa.core.policies.policy import Policy
from rasa.core.policies.registry import FallbackPolicy, KerasPolicy, MemoizationPolicy, FormPolicy, MappingPolicy
from rasa.core.agent import Agent

import asyncio

# this will catch predictions the model isn't very certain about
# there is a threshold for the NLU predictions as well as the action predictions
fallback = FallbackPolicy(fallback_action_name="utter_unclear",
                          core_threshold=0.2,
                          nlu_threshold=0.1)

agent = Agent('domain-papaito.yml',
              policies=[MappingPolicy(),
                        MemoizationPolicy(max_history=5),
                        KerasPolicy(validation_split=0.0,epochs=400),
                        FormPolicy(),
                        fallback])

# loading our neatly defined training dialogues
training_data = await agent.load_data('data/core/stories-papaito.md')

agent.train(training_data)

agent.persist('models/dialogue')

Processed trackers: 100%|██████████| 28/28 [00:00<00:00, 618.34it/s, # actions=76]
Processed actions: 76it [00:00, 2171.45it/s, # examples=72]
Processed trackers:  52%|█████▏    | 275/528 [00:01<00:00, 256.51it/s, # actions=892]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1430/1430 [==============================] - 2s 1ms/sample - loss: 3.2179 - accuracy: 0.3951
Epoch 2/400
1430/1430 [==============================] - 0s 95us/sample - loss: 2.4443 - accuracy: 0.4580
Epoch 3/400
1430/1430 [==============================] - 0s 94us/sample - loss: 2.2989 - accuracy: 0.4580
Epoch 4/400
1430/1430 [==============================] - 0s 90us/sample - loss: 2.2226 - accuracy: 0.4580
Epoch 5/400
1430/1430 [==============================] - 0s 96us/sample - loss: 2.1580 - accuracy: 0.4580
Epoch 6/400
1430/1430 [==============================] - 0s 105us/sample - loss: 2.0860 - accuracy: 0.4580
Epoch 7/400
1430/1430 [==============================] - 0s 105us/sample - loss: 2.0176 - accuracy: 0.4580
Epoch 8/400
1430/1430 [==============================] - 0s 105us/sample - loss: 1.9376 - accuracy: 0.4580
Epoch 9/400
1430/1430 [==============================] - 0s 110us/sample - loss: 1.8370 - accuracy: 0.4587
Epoch 10/400
1430/1430 [==============================]

1430/1430 [==============================] - 0s 104us/sample - loss: 0.2320 - accuracy: 0.9203
Epoch 78/400
1430/1430 [==============================] - 0s 98us/sample - loss: 0.2159 - accuracy: 0.9343
Epoch 79/400
1430/1430 [==============================] - 0s 102us/sample - loss: 0.2543 - accuracy: 0.9182
Epoch 80/400
1430/1430 [==============================] - 0s 109us/sample - loss: 0.2453 - accuracy: 0.9224
Epoch 81/400
1430/1430 [==============================] - 0s 109us/sample - loss: 0.2226 - accuracy: 0.9266
Epoch 82/400
1430/1430 [==============================] - 0s 101us/sample - loss: 0.2402 - accuracy: 0.9308
Epoch 83/400
1430/1430 [==============================] - 0s 91us/sample - loss: 0.2441 - accuracy: 0.9203
Epoch 84/400
1430/1430 [==============================] - 0s 91us/sample - loss: 0.2276 - accuracy: 0.9259
Epoch 85/400
1430/1430 [==============================] - 0s 91us/sample - loss: 0.2416 - accuracy: 0.9210
Epoch 86/400
1430/1430 [=====================

1430/1430 [==============================] - 0s 94us/sample - loss: 0.2274 - accuracy: 0.9210
Epoch 153/400
1430/1430 [==============================] - 0s 108us/sample - loss: 0.2173 - accuracy: 0.9252
Epoch 154/400
1430/1430 [==============================] - 0s 107us/sample - loss: 0.1973 - accuracy: 0.9357
Epoch 155/400
1430/1430 [==============================] - 0s 109us/sample - loss: 0.2206 - accuracy: 0.9231
Epoch 156/400
1430/1430 [==============================] - 0s 93us/sample - loss: 0.2279 - accuracy: 0.9224
Epoch 157/400
1430/1430 [==============================] - 0s 86us/sample - loss: 0.2249 - accuracy: 0.9224
Epoch 158/400
1430/1430 [==============================] - 0s 97us/sample - loss: 0.2481 - accuracy: 0.9147
Epoch 159/400
1430/1430 [==============================] - 0s 98us/sample - loss: 0.2257 - accuracy: 0.9224
Epoch 160/400
1430/1430 [==============================] - 0s 97us/sample - loss: 0.1936 - accuracy: 0.9357
Epoch 161/400
1430/1430 [==============

1430/1430 [==============================] - 0s 94us/sample - loss: 0.2009 - accuracy: 0.9273
Epoch 228/400
1430/1430 [==============================] - 0s 93us/sample - loss: 0.1932 - accuracy: 0.9336
Epoch 229/400
1430/1430 [==============================] - 0s 98us/sample - loss: 0.2142 - accuracy: 0.9224
Epoch 230/400
1430/1430 [==============================] - 0s 93us/sample - loss: 0.1981 - accuracy: 0.9308
Epoch 231/400
1430/1430 [==============================] - 0s 93us/sample - loss: 0.1791 - accuracy: 0.9350
Epoch 232/400
1430/1430 [==============================] - 0s 94us/sample - loss: 0.2130 - accuracy: 0.9273
Epoch 233/400
1430/1430 [==============================] - 0s 95us/sample - loss: 0.1915 - accuracy: 0.9301
Epoch 234/400
1430/1430 [==============================] - 0s 95us/sample - loss: 0.2088 - accuracy: 0.9259
Epoch 235/400
1430/1430 [==============================] - 0s 94us/sample - loss: 0.1982 - accuracy: 0.9224
Epoch 236/400
1430/1430 [=================

1430/1430 [==============================] - 0s 94us/sample - loss: 0.1888 - accuracy: 0.9308
Epoch 303/400
1430/1430 [==============================] - 0s 95us/sample - loss: 0.2069 - accuracy: 0.9231
Epoch 304/400
1430/1430 [==============================] - 0s 93us/sample - loss: 0.1747 - accuracy: 0.9371
Epoch 305/400
1430/1430 [==============================] - 0s 95us/sample - loss: 0.1768 - accuracy: 0.9343
Epoch 306/400
1430/1430 [==============================] - 0s 94us/sample - loss: 0.2070 - accuracy: 0.9252
Epoch 307/400
1430/1430 [==============================] - 0s 93us/sample - loss: 0.2359 - accuracy: 0.9133
Epoch 308/400
1430/1430 [==============================] - 0s 95us/sample - loss: 0.1914 - accuracy: 0.9385
Epoch 309/400
1430/1430 [==============================] - 0s 96us/sample - loss: 0.2123 - accuracy: 0.9231
Epoch 310/400
1430/1430 [==============================] - 0s 93us/sample - loss: 0.2067 - accuracy: 0.9273
Epoch 311/400
1430/1430 [=================

1430/1430 [==============================] - 0s 99us/sample - loss: 0.1719 - accuracy: 0.9364
Epoch 378/400
1430/1430 [==============================] - 0s 90us/sample - loss: 0.1729 - accuracy: 0.9420
Epoch 379/400
1430/1430 [==============================] - 0s 92us/sample - loss: 0.1846 - accuracy: 0.9371
Epoch 380/400
1430/1430 [==============================] - 0s 90us/sample - loss: 0.1850 - accuracy: 0.9329
Epoch 381/400
1430/1430 [==============================] - 0s 91us/sample - loss: 0.1990 - accuracy: 0.9273
Epoch 382/400
1430/1430 [==============================] - 0s 91us/sample - loss: 0.2204 - accuracy: 0.9203
Epoch 383/400
1430/1430 [==============================] - 0s 92us/sample - loss: 0.1791 - accuracy: 0.9301
Epoch 384/400
1430/1430 [==============================] - 0s 91us/sample - loss: 0.1830 - accuracy: 0.9315
Epoch 385/400
1430/1430 [==============================] - 0s 92us/sample - loss: 0.1895 - accuracy: 0.9273
Epoch 386/400
1430/1430 [=================

Processed trackers: 100%|██████████| 28/28 [00:00<00:00, 646.96it/s, # actions=73]
INFO:rasa.core.agent:Model directory models/dialogue/core exists and contains old model files. All files will be overwritten.
INFO:rasa.core.agent:Persisted model to '/home/debian/workspace/Dadbot/models/dialogue/core'


# Talk to your Bot

In [12]:
#Starting the Bot
from rasa.core.agent import Agent
from rasa.core.utils import EndpointConfig

action_endpoint = EndpointConfig(url="http://0.0.0.0:5055/webhook")
agent = Agent.load('models/dialogue', interpreter=model_directory, action_endpoint=action_endpoint)

#!docker run -d -p 5055:5055 --mount type=bind,source=/home/debian/workspace/Dadbot/actions/actions.py,target=/app/actions rasa/rasa-sdk

## Creating the voice synthesizer

In [13]:
#!git clone https://github.com/NVIDIA/tacotron2.git

from tacotron2.hparams import create_hparams
from tacotron2.model import Tacotron2
from tacotron2.layers import TacotronSTFT, STFT
from tacotron2.audio_processing import griffin_lim
from tacotron2.train import load_model
from tacotron2.text import text_to_sequence
from tacotron2.waveglow.mel2samp import files_to_list, MAX_WAV_VALUE
from tacotron2.denoiser import Denoiser
import numpy as np
import torch

def synthesize(text, voice, sigma=0.6, denoiser_strength=0.05, is_fp16=False):

    hparams = create_hparams()
    hparams.sampling_rate = 22050

    if voice == "papaito":
        voice_model = "nvidia_tacotron2_papaito_300"
    elif voice == "constantino":
        voice_model = "tacotron2_Constantino_600"
    elif voice == "orador":
        voice_model = "checkpoint_tacotron2_29000_es"
    
    checkpoint_path = "/home/debian/workspace/models/" + voice_model
        
    model = load_model(hparams)
    model.load_state_dict(torch.load(checkpoint_path)['state_dict'])
    _ = model.cuda().eval().half()

    waveglow_path = '/home/debian/workspace/models/waveglow_256channels_ljs_v2.pt'
    waveglow = torch.load(waveglow_path)['model']
    _ = waveglow.cuda().eval().half()
    denoiser = Denoiser(waveglow)

    #text="¡Cágate lorito!"
    #with open(filelist_path, encoding='utf-8', mode='r') as f:
    #    text = f.read()

    sequence = np.array(text_to_sequence(text, ['english_cleaners']))[None, :]
    sequence = torch.autograd.Variable(
        torch.from_numpy(sequence)).cuda().long()

    mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
    #mel = torch.unsqueeze(mel, 0)
    mel = mel_outputs.half() if is_fp16 else mel_outputs
    audio = np.array([])
    with torch.no_grad():
        audio = waveglow.infer(mel, sigma=sigma)
        if denoiser_strength > 0:
             audio = denoiser(audio, denoiser_strength)
        audio = audio * MAX_WAV_VALUE
        audio = audio.squeeze()
        audio = audio.cpu().numpy()
        audio = audio.astype('int16')
    
    return audio, hparams.sampling_rate

## Start chat

In [16]:
import sounddevice as sd
from sty import fg, bg, ef, rs

print("¡Da-bot está listo para cascar! Escribe tus mensajes o dile 'quieto parao'")
while True:
    a = input()
    
    if a == 'quieto parao':
        break
    responses = await agent.handle_text(a)
    for response in responses:
        to_synth = response["text"]
        #to_synth = "Esto es una prueba para ver si funciona"
        print(fg.blue + to_synth + fg.rs)
        response_file = open('response.txt','w') 
        response_file.write(to_synth)
        voice, sr = synthesize(to_synth, "orador")
        sd.play(voice, sr)
        response_file.close()        

¡Da-bot está listo para cascar! Escribe tus mensajes o dile 'quieto parao'
hola
¿Qué tal?


AttributeError: module 'tensorflow' has no attribute 'contrib'